# Koble til base
Tips: https://www.geeksforgeeks.org/how-to-write-pandas-dataframes-to-multiple-excel-sheets/

In [5]:
import pandas as pd
import sqlite3
# Connect to the SQLite database
def getConn():
    conn = sqlite3.connect("C:/Projects/nin3-kode-api/NiN3.WebApi/nin3kodeapi.db")
    return conn


# Type-fanen

In [16]:
conn = getConn()
TypefaneQ = """WITH eco AS (
        SELECT Ordinal, Verdi, Beskrivelse 
        FROM Enumoppslag 
        WHERE Enumtype = 'EcosysnivaaEnum'
    ),
    tkat AS (
        SELECT Ordinal, Verdi, Beskrivelse
        FROM Enumoppslag
        WHERE Enumtype = 'TypekategoriEnum'
    ),
    tkat2 AS ( 
        SELECT Ordinal, Verdi, Beskrivelse
        FROM Enumoppslag
        WHERE Enumtype = 'Typekategori2Enum'
    )
    SELECT Langkode, Kode, type.Navn, eco.Verdi AS EcosystnivaaEnum, tkat.Verdi AS Typekategori, tkat2.Verdi AS Typekategori2
    FROM type
    LEFT JOIN eco ON type.Ecosystnivaa = eco.Ordinal
    LEFT JOIN tkat ON type.Typekategori = tkat.Ordinal
    LEFT JOIN tkat2 ON type.Typekategori2 = tkat2.Ordinal"""  # Write your SQLite query
df_type = pd.read_sql_query(TypefaneQ, conn)  # Execute the query and store the result in a DataFrame
#Ecosystnivå
EcoQ = "Select Verdi, Beskrivelse from Enumoppslag where Enumtype = 'EcosysnivaaEnum'"
df_eco = pd.read_sql_query(EcoQ, conn)
conn.close()

## Hovedtypegruppe-fanen

In [23]:
#HTG
conn = getConn()
htgQ = """WITH tkat2 AS ( 
        SELECT Ordinal, Verdi, Beskrivelse
        FROM Enumoppslag
        WHERE Enumtype = 'Typekategori2Enum'
    ), tkat3 AS ( 
        SELECT Ordinal, Verdi, Beskrivelse
        FROM Enumoppslag
        WHERE Enumtype = 'Typekategori3Enum'
    )
    SELECT htg.Langkode, htg.Kode, htg.Delkode, htg.Navn, tkat2.Verdi AS Typekategori2, tkat3.Verdi AS Typekategori3, type.Kode AS Typekode
    FROM hovedtypegruppe htg
    LEFT JOIN tkat2 ON htg.Typekategori2 = tkat2.Ordinal
    LEFT JOIN tkat3 ON htg.Typekategori3 = tkat3.Ordinal
    LEFT JOIN type ON htg.TypeId = type.Id"""
df_htg = pd.read_sql_query(htgQ, conn) 
display(df_htg.head(3))
conn.close()

,Langkode,Kode,Delkode,Navn,Typekategori2,Typekategori3,Typekode
0,NIN-3.0-T-A-LV-FL-0-FL-A,FL-A,A,Breelvlandformer,FL,None,A-LV-FL
1,NIN-3.0-T-A-LV-FL-0-FL-B,FL-B,B,Biogene landformer,FL,None,A-LV-FL
2,NIN-3.0-T-A-LV-FL-0-FL-C,FL-C,C,Bresjølandformer,FL,None,A-LV-FL


## Hovedtype-fanen

In [29]:
conn = getConn()
htQ = """WITH pk AS ( 
    SELECT Ordinal, Verdi, Beskrivelse
    FROM Enumoppslag
    WHERE Enumtype = 'ProsedyrekategoriEnum'
    )
    SELECT ht.Langkode, ht.Kode, ht.Delkode, ht.Navn, pk.Verdi AS Prosedyrekategori, hg.Kode AS Hovedtypegruppekode
    FROM hovedtype ht
    LEFT JOIN hovedtypegruppe hg ON hg.Id = ht.HovedtypegruppeId
    LEFT JOIN pk ON ht.Prosedyrekategori = pk.Ordinal"""
df_ht = pd.read_sql_query(htQ, conn)
conn.close()

## Grunntype-fanen

In [31]:
conn = getConn()
gtQ = """WITH pk AS ( 
    SELECT Ordinal, Verdi, Beskrivelse
    FROM Enumoppslag
    WHERE Enumtype = 'ProsedyrekategoriEnum'
    )
    Select gt.Langkode, gt.Kode, gt.Delkode, gt.Navn, pk.Verdi AS Prosedyrekategori, ht.Kode AS Hovedtypekode
    from grunntype gt
    LEFT JOIN pk ON gt.Prosedyrekategori = pk.Ordinal
    LEFT JOIN hovedtype ht ON gt.HovedtypeId = ht.Id"""
df_gt = pd.read_sql_query(gtQ, conn)
df_gt.head(3)
conn.close()  

In [30]:
#quick view of table
conn = getConn()
tablename = 'Grunntype'
query = "select * from " + tablename
df = pd.read_sql_query(query, conn)
display(df.head(3))
conn.close()

,Id,VersjonId,Langkode,Kode,Navn,Delkode,Prosedyrekategori,HovedtypeId,KartleggingsenhetId
0,1,1,NIN-3.0-T-C-PE-LA-0-I-0-01-001,I-0-01-001,Dalformet lavlands-åslandskap,001,NaN,238,None
1,2,1,NIN-3.0-T-C-PE-LA-0-I-0-01-002,I-0-01-002,Dalformet lavlands-åslandskap med jordbruk,002,NaN,238,None
2,3,1,NIN-3.0-T-C-PE-LA-0-I-0-01-003,I-0-01-003,Dalformet lavlands-åslandskap med bygd/bebyggelse,003,NaN,238,None


In [3]:
## opprett excel
with pd.ExcelWriter("nin3_0.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    df_type.to_excel(writer, sheet_name="Type", index=False)
    df_htg.to_excel(writer, sheet_name="Hovedtypegruppe", index=False)
    df_ht.to_excel(writer, sheet_name="Hovedtype", index=False)
    df_gt.to_excel(writer, sheet_name="Grunntype", index=False)
    df_eco.to_excel(writer, sheet_name="Ecosystnivå", index=False)
    #data_frame3.to_excel(writer, sheet_name="Baked Items", index=False)
    

In [4]:
from datetime import datetime
import sqlite3
from datetime import datetime

current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
print(current_time)


# Connect to the SQLite database
conn = sqlite3.connect("C:/Projects/nin3-kode-api/NiN3.WebApi/nin3kodeapi.db")

# Get the current timestamp
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

# Insert the values into the db_info table
insert_query = "INSERT INTO db_info (Tittel, Verdi) VALUES (?, ?)"
conn.execute(insert_query, ('Opprettet', current_time))

# Commit the changes and close the connection
conn.commit()
conn.close()
# Connect to the SQLite database
conn = sqlite3.connect("your_database.db")

# Get the current timestamp
current_time = conn.execute("SELECT ").fetchone()[0]
strftime('%Y%m%d_%H%M%S', 'now', 'localtime')
# Print the current time
print(current_time)

# Close the connection
conn.close()


20231129_091209


In [2]:
import os
current_directory = os.getcwd()
print(current_directory)


AttributeError: module 'os' has no attribute 'cwd'